# **DisplaCy visualiser tool for the metadata annotations**

In [2]:
# Imports
import json
from spacy import displacy
import re
import os

Currently the annotations structure is as follows:

```python
start_index = int # The index of the first character of the entity. 
end_index = int # The index of the last character of the entity. (not inclusive)
label = str # Label attached to the entity. 

{
    "classes": ["TEMP", "SOFT", "STIME", "MOL", "FFM"],
    "annotations": [
        [
            "text_to_fill_in",
            {
                "entities": [
                    [start_index, end_index, label],
                ]
            }
        ]
    ]
}

# Example
{
    "classes": ["TEMP", "SOFT", "STIME", "MOL", "FFM"],
    "annotations": [
        [
            "POPC Ulmschneider OPLS Verlet Group",
            {
                "entities": [
                    [0, 4, "MOL"],
                    [18, 22, "FFM"]
                ]
            }
        ]
    ]
}

```


However, the spaCy  format for `ent` is as follows:

```python
start_index = int # The index of the first character of the entity. 
end_index = int # The index of the last character of the entity. (not inclusive)
label = str # Label attached to the entity. 

{
  "text": "text_to_fill_in.",
  "ents": [{ "start": start_index, "end": end_index, "label": label }]
}

# Example:
{
  "text": "POPC molecules.",
  "ents": [{ "start": 0, "end": 4, "label": "MOL" }]
}

```
This is what it looks like working with the spaCy format:

In [3]:
text = "Marie Curie was born in Warsaw in 1867 and later founded the Radium Institute in Paris"
ents = [
    {"start": 0, "end": 11, "label": "PERS"},
    {"start": 24, "end": 30, "label": "LOC"},
    {"start": 34, "end": 38, "label": "DATE"},
    {"start": 61, "end": 77, "label": "ORG"},
    {"start": 81, "end": 86, "label": "LOC"},
]

data = {"text": text, "ents": ents}

colors = {
    "PERS": "#a8ff65",
    "LOC": "orange",
    "DATE": "turquoise",
    "ORG": "#7361ff",
}
options = {"colors": colors}

displacy.render(data, style="ent", manual=True, options=options)

However we don't have this kind of format. 
We need to therefore convert the ground truth format to something spaCy-compatible.

Let's do this for ground truth texts:

In [4]:
def convert_annotations(file_path):
    """
    Convert annotations from the custom format to spaCy's displaCy format.

    Parameters:
        file_path (str): Path to the JSON file containing "classes" and "annotations" keys.

    Returns:
        List[dict]: A list of dictionaries in the spaCy format.
    """
    with open(file_path, "r") as file:
        data = json.load(file)

    converted = []
    for item in data["annotations"]:
        # Each item is structured as [text, {"entities": [[start, end, label], ...]}]
        text, annotation_info = item
        entities = annotation_info.get("entities", [])
        # Convert each entity to the spaCy dictionary format
        ents = [
            {"start": start, "end": end, "label": label}
            for start, end, label in entities
        ]
        converted.append({"text": text, "ents": ents})
    return converted

In [5]:
# Here we take one example of ground-truth annotation.
data = "../annotations/zenodo_3780463.json"

# Convert the annotations.
converted_data = convert_annotations(data)
original_text = converted_data[0]["text"]

# Print the original text, but keep the format with the \n characters.
# print(repr(original_text))

colors = {
    "TEMP": "#ffb3ba",
    "SOFT": "#ffffba",
    "STIME": "#baffc9",
    "MOL": "#bae1ff",
    "FFM": "#cdb4db",
}
options = {"colors": colors}

# You now have a list of annotation dictionaries. For visualization with displaCy,
# you can serve a single annotated text or multiple.
displacy.render(converted_data, style="ent", manual=True, options=options)

Let's say we have a ground-truth annotation that we want to use as an example in a prompt.
We'll first display the input text so that we can read it properly, then we use spaCy to visualise the ground-truth annotation, and lastly, we modify the annotation format in order to get a text that we can use in prompts.

In this following case, we want the ground-truth annotation modified to an annotation using XML-style tags.

In [6]:
def modify_annotation(text_path: str):
    # Convert the annotations.
    converted_data = convert_annotations(data)
    original_text = converted_data[0]["text"]

    # Print the original text, but keep the format with the \n characters.
    print(repr(original_text), "\n")

    colors = {
        "TEMP": "#ffb3ba",
        "SOFT": "#ffffba",
        "STIME": "#baffc9",
        "MOL": "#bae1ff",
        "FFM": "#cdb4db",
    }
    options = {"colors": colors}

    # You now have a list of annotation dictionaries. For visualization with displaCy,
    # you can serve a single annotated text or multiple.
    displacy.render(converted_data, style="ent", manual=True, options=options)

    # 1. Sort entities by start offset (ascending)
    ents = sorted(converted_data[0]["ents"], key=lambda e: e["start"])
    pieces = []
    last_idx = 0

    # 2. For each entity, append:
    #    – the text from the end of the previous entity (or start of string)
    #    – the wrapped entity
    for e in ents:
        start, end, label = e["start"], e["end"], e["label"]
        # plain text before this entity
        pieces.append(original_text[last_idx:start])
        # the entity wrapped in tags
        entity_text = original_text[start:end]
        pieces.append(f"<{label}>{entity_text}</{label}>")
        last_idx = end

    # 3. Append any trailing text after the last entity
    pieces.append(original_text[last_idx:])
    return repr("".join(pieces))

In [16]:
data = "../annotations/zenodo_4300706.json"

# Top we will have the input text
# Middle we will have the spaCy visualisation
# Last we will have the modified annotation using XML-style tags
print(modify_annotation(data))

'256 DPPC Molecules bilayer in pure Water, simulated at 288K (gel) or 358K (fluid)\nPublication: MLLPA: A Machine Learning-assisted Python module to study phase-specific events in lipid membranes Published on: 08 April 2021 Journal: J Comp Chem, 2021, DOI: 10.1002/jcc.26508 Description: Simulation files used to train our Python module to identify the thermodynamic phase of individual lipid molecules in a bilayer, as well as the simulation files analysed by the machine learning models. More information on the module can be found on its website. The training files are named dppc gel.gro and dppc fluid.gro. They respectively correspond to the final frame of the systems simulated at 288K and 358K. All other files are the files analysed by the module. System composition: DPPC molecules: 256 with 130 atoms each Water molecules: 42,492 with 3 atoms each Simulation box dimensions (approx.): 9 x 9 x 20 nm Simulation details: Software: Gromacs (v. 2020) Forcefield: Charmm36 (v. June 2015) - Wate

'256 <MOL>DPPC</MOL> Molecules bilayer in pure <MOL>Water</MOL>, simulated at <TEMP>288K</TEMP> (gel) or <TEMP>358K</TEMP> (fluid)\nPublication: MLLPA: A Machine Learning-assisted Python module to study phase-specific events in lipid membranes Published on: 08 April 2021 Journal: J Comp Chem, 2021, DOI: 10.1002/jcc.26508 Description: Simulation files used to train our Python module to identify the thermodynamic phase of individual lipid molecules in a bilayer, as well as the simulation files analysed by the machine learning models. More information on the module can be found on its website. The training files are named <MOL>dppc</MOL> gel.gro and <MOL>dppc</MOL> fluid.gro. They respectively correspond to the final frame of the systems simulated at <TEMP>288K</TEMP> and <TEMP>358K.</TEMP> All other files are the files analysed by the module. System composition: <MOL>DPPC</MOL> molecules: 256 with 130 atoms each <MOL>Water</MOL> molecules: 42,492 with 3 atoms each Simulation box dimensio

### **LLM annotations compared to GT annotations**
---

Here we give the functions to adapt the following formats into the spaCy format:
- XML-style labels
- JSON output
- Index based

In [22]:
def convert_xml_to_spacy_format(annotated_text):
    with open(annotated_text, "r") as file:
        annotated_text = file.read()

    pattern = re.compile(r"<(MOL|STIME|TEMP|SOFTNAME|SOFTVERS|FFM)>(.+?)<\1>")
    ents = []
    clean_text = ""
    last_idx = 0  # Position in the annotated text

    for match in pattern.finditer(annotated_text):
        tag = match.group(1)
        entity_text = match.group(2)

        # Text between last entity and current
        pre_entity_text = annotated_text[last_idx : match.start()]
        clean_text += pre_entity_text
        entity_start = len(clean_text)
        clean_text += entity_text
        entity_end = len(clean_text)

        ents.append({"start": entity_start, "end": entity_end, "label": tag})

        last_idx = match.end()

    # Add any remaining text after last tag
    clean_text += annotated_text[last_idx:]

    return [{"text": clean_text, "ents": ents}]

In [26]:
def visualize_xml_vs_gt(data: list) -> None:
    """
    Visualize the annotated text using displaCy.

    Parameters:
        data (list): List of paths to the annotated text files.
    """
    # Convert the annotations.
    converted_real_data = convert_annotations(data[0])  # Real annotation
    converted_ai_data = convert_xml_to_spacy_format(data[1])  # AI annotated

    colors = {
        "TEMP": "#ffb3ba",
        "SOFTNAME": "#ffffba",
        "SOFTVERS": "#orange",
        "STIME": "#baffc9",
        "MOL": "#bae1ff",
        "FFM": "#cdb4db",
    }
    options = {"colors": colors}

    print("\nREAL ANNOTATION:")
    displacy.render(converted_real_data, style="ent", manual=True, options=options)
    print("\AI XML ANNOTATION:")
    displacy.render(converted_ai_data, style="ent", manual=True, options=options)

In [ ]:
# ai_path = "../llm_outputs/annotations_2025-04-22_17-31-26/few_shot/gpt-4.1-2025-04-14/zenodo_7007107.json"
# ai_path = "../llm_outputs/annotations_2025-04-22_17-31-26/few_shot/gpt-4.1-2025-04-14/zenodo_1219494.json"
# ai_path = "../llm_outputs/annotations_2025-04-22_17-31-26/one_shot/gpt-4.1-2025-04-14/zenodo_7323535.json"
# ai_path = "../output_llm_annotations/zero_shot_prompt_template/deepseek-r1-distill-llama-70b/figshare_20300547.json"

# zenodo_6478270
# zenodo_3778112


# o3-2025-04-16
# o3-mini-2025-01-31
# o4-mini-2025-04-16
ai_path = "../llm_outputs/2025-04-28_17-32-24/annotations/few_shot/2025-05-27_14-50-19/zenodo_3780463.json"
real_path = "../annotations/zenodo_3780463.json"

# ai_path = "../testing/ai_text.json"
# real_path = "../testing/real_text.json"

converted_real_data = convert_annotations(real_path)  # Real annotation


def convert_ai_xml(annotated_text):
    with open(annotated_text, "r") as file:
        annotated_text = json.load(file)

    annotated_text = annotated_text["response"]

    pattern = re.compile(r"<(MOL|STIME|TEMP|SOFTNAME|SOFTVERS|FFM)>(.+?)</\1>")
    ents = []
    clean_text = ""
    last_idx = 0  # Position in the annotated text

    for match in pattern.finditer(annotated_text):
        tag = match.group(1)
        entity_text = match.group(2)

        # Text between last entity and current
        pre_entity_text = annotated_text[last_idx : match.start()]
        clean_text += pre_entity_text
        entity_start = len(clean_text)
        clean_text += entity_text
        entity_end = len(clean_text)

        ents.append({"start": entity_start, "end": entity_end, "label": tag})

        last_idx = match.end()

    # Add any remaining text after last tag
    clean_text += annotated_text[last_idx:]
    ents = []
    return [{"text": clean_text, "ents": ents}]


converted_ai_text = convert_ai_xml(ai_path)

colors = {
    "TEMP": "#ffb3ba",
    "SOFTNAME": "#ffffba",
    "SOFTVERS": "#orange",
    "STIME": "#baffc9",
    "MOL": "#bae1ff",
    "FFM": "#cdb4db",
}
options = {"colors": colors}

displacy.render(converted_real_data, style="ent", manual=True, options=options)

displacy.render(converted_ai_text, style="ent", manual=True, options=options)


Using the 'ent' visualizer
Serving on http://0.0.0.0:5000 ...



127.0.0.1 - - [19/Jun/2025 13:49:17] "GET / HTTP/1.1" 200 662


Shutting down server on port 5000.


In [ ]:
real_data = "../annotations/zenodo_3950029.json"
xml_ai_data = "../ai_annotations/chatgpt_zenodo_3950029.txt"

data = [real_data, xml_ai_data]

visualize_xml_vs_gt(data)


REAL ANNOTATION:



AI ANNOTATION:


### **REAL ANNOTATION**

In [ ]:
real_data = "../annotations/zenodo_1346073.json"

# Convert the annotations.
converted_data = convert_annotations(real_data)

colors = {
    "TEMP": "#ffb3ba",
    "SOFTNAME": "#ffffba",
    "SOFTVERS": "#orange",
    "STIME": "#baffc9",
    "MOL": "#bae1ff",
    "FFM": "#cdb4db",
}
options = {"colors": colors}

# You now have a list of annotation dictionaries. For visualization with displaCy,
# you can serve a single annotated text or multiple.
displacy.render(converted_data, style="ent", manual=True, options=options)

### **A.I. ANNOTATIONS**

In [ ]:
ai_data = "../ai_annotations/llama_3_1_b_annotation.txt"

converted_ai_data = convert_annotated_text_to_spacy_format(ai_data)

colors = {
    "TEMP": "#ffb3ba",
    "SOFTNAME": "#ffffba",
    "SOFTVERS": "#orange",
    "STIME": "#baffc9",
    "MOL": "#bae1ff",
    "FFM": "#cdb4db",
}
options = {"colors": colors}

print("\nAI ANNOTATION:")
displacy.render(converted_ai_data, style="ent", manual=True, options=options)


AI ANNOTATION:


### **All ground-truth annotations visualised using spaCy**
---

In [7]:
folder = "../annotations"

for filename in os.listdir(folder):
    # if ends with json and contains one '_' only
    if filename.endswith(".json") and filename.count("_") == 1:
        file_path = os.path.join(folder, filename)
        converted_data = convert_annotations(file_path)
        colors = {
            "TEMP": "#ffb3ba",
            "SOFTNAME": "#ffffba",
            "SOFTVERS": "#orange",
            "STIME": "#baffc9",
            "MOL": "#bae1ff",
            "FFM": "#cdb4db",
        }
        options = {"colors": colors}
        print(f"Visualizing {filename}...")
        displacy.render(converted_data, style="ent", manual=True, options=options)
        print("\n=================================================================\n")

Visualizing figshare_22213635.json...




Visualizing figshare_4757161.json...




Visualizing figshare_21263177.json...




Visualizing zenodo_6582985.json...




Visualizing zenodo_6478270.json...


...



Visualizing figshare_12312368.json...




Visualizing figshare_12120159.json...




Visualizing zenodo_30904.json...




Visualizing figshare_9805616.json...




Visualizing osf_82n73.json...


In [13]:
from pathlib import Path
from typing import Dict, Any, List


def convert_label_annotations(file_path: str | Path) -> Dict[str, Any]:
    """
    Convert one custom-format JSON file to the structure required by
    spaCy's displaCy manual visualiser.

    Returns
    -------
    {
        "text": "<full text>",
        "ents": [ {"start": 0, "end": 5, "label": "TAG"}, ... ]
    }
    """
    file_path = Path(file_path)

    with file_path.open(encoding="utf-8") as f:
        data = json.load(f)

    ents: List[Dict[str, Any]] = [
        {"start": int(s), "end": int(e), "label": tag} for s, e, tag in data["label"]
    ]

    return {"text": data["text"], "ents": ents}

In [17]:
original_annotations = "../annotations/"
converted_annotations = "../testing/annotations_attempts/"

for filename in os.listdir(original_annotations):
    # if ends with json and contains one '_' only
    if filename.endswith(".json") and filename.count("_") == 1:
        file_path = os.path.join(original_annotations, filename)
        converted_file_path = os.path.join(converted_annotations, filename)

        converted_original_data = convert_annotations(file_path)
        converted_label_data = convert_label_annotations(converted_file_path)

        colors = {
            "TEMP": "#ffb3ba",
            "SOFTNAME": "#ffffba",
            "SOFTVERS": "#orange",
            "STIME": "#baffc9",
            "MOL": "#bae1ff",
            "FFM": "#cdb4db",
        }
        options = {"colors": colors}
        displacy.render(
            converted_original_data, style="ent", manual=True, options=options
        )
        print("----")
        displacy.render(converted_label_data, style="ent", manual=True, options=options)
        print("\n=================================================================\n")

NameError: name 'os' is not defined